In [22]:
import pandas as pd

overdose = pd.read_csv('Overdose.csv')

#Filter for years after or at 2018. Removing the row containing the population for each
#county per year in order to avoid counting it towards the overdose count
mask = (overdose['Year'] >= 2018) & (overdose['Source']!= 'Population')
overdose = overdose[mask]

#Overdoses by years 2018, 2019, 2020
overdose = overdose.groupby(['County', 'Year'])["Count"].sum().reset_index()
overdose = overdose.rename(columns={'Count':'Overdoses'})
year18 = overdose[overdose["Year"] == 2018] 
year19 = overdose[overdose["Year"] == 2019] 
year20 = overdose[overdose["Year"] == 2020] 
overdose['2018 Overdoses'] = year18["Overdoses"]
overdose['2019 Overdoses'] = year19["Overdoses"]
overdose['2020 Overdoses'] = year20["Overdoses"]
overdose = overdose.drop(columns = ["Year", "Overdoses"]).groupby("County").sum().reset_index()
overdose.head(100)

,County,2018 Overdoses,2019 Overdoses,2020 Overdoses
0,Alamance,2263.0,2231.0,2351.0
1,Alexander,642.0,432.0,459.0
2,Alleghany,137.0,120.0,165.0
3,Anson,459.0,303.0,444.0
4,Ashe,370.0,347.0,322.0
...,...,...,...,...
95,Wayne,1613.0,2054.0,1658.0
96,Wilkes,1356.0,1158.0,1044.0
97,Wilson,817.0,988.0,1316.0
98,Yadkin,767.0,544.0,607.0


In [23]:
income = pd.read_csv('income.csv')

income = income.rename(columns={'Region': 'County', '2018' : '2018 Per Capita Income', '2019' : '2019 Per Capita Income', '2020' : '2020 Per Capita Income'})
income.head(100)

,County,2018 Per Capita Income,2019 Per Capita Income,2020 Per Capita Income
0,Alamance,39742,41256,43973
1,Alexander,36539,37758,40176
2,Alleghany,36576,37987,40599
3,Anson,35135,37741,38785
4,Ashe,34791,36643,39612
...,...,...,...,...
95,Wayne,38115,40116,42882
96,Wilkes,36445,37309,39518
97,Wilson,39158,40260,43111
98,Yadkin,37328,39012,41250


In [24]:
#merge together for our combined dataset
data = pd.merge(overdose, income, on = "County")

#Convert float to int
data['2018 Overdoses'] = data['2018 Overdoses'].astype(int)
data['2019 Overdoses'] = data['2019 Overdoses'].astype(int)
data['2020 Overdoses'] = data['2020 Overdoses'].astype(int)

data.head(100)

,County,2018 Overdoses,2019 Overdoses,2020 Overdoses,2018 Per Capita Income,2019 Per Capita Income,2020 Per Capita Income
0,Alamance,2263,2231,2351,39742,41256,43973
1,Alexander,642,432,459,36539,37758,40176
2,Alleghany,137,120,165,36576,37987,40599
3,Anson,459,303,444,35135,37741,38785
4,Ashe,370,347,322,34791,36643,39612
...,...,...,...,...,...,...,...
95,Wayne,1613,2054,1658,38115,40116,42882
96,Wilkes,1356,1158,1044,36445,37309,39518
97,Wilson,817,988,1316,39158,40260,43111
98,Yadkin,767,544,607,37328,39012,41250


In [25]:
population = pd.read_csv('county_estimates_2020.csv', sep = ";")
population = population.drop(columns = ["Estimate", "geom", "centroid"])
population["Year"] = pd.DatetimeIndex(population["Year"]).year
population = population.sort_values(["County", "Year"]).reset_index().drop(columns = "index")


#If year and county is the same, take the latest reported data for that year
population = population.drop_duplicates(subset = ['County','Year'], keep = 'last')

#rearrangement
population = population[population["Year"] >= 2018] 
year18 = population[population["Year"] == 2018] 
year19 = population[population["Year"] == 2019] 
year20 = population[population["Year"] == 2020] 

population = population.drop(columns = "Year")
population['2018 Population'] = year18["value"]
population['2019 Population'] = year19["value"]
population['2020 Population'] = year20["value"]
population = population.drop(columns = "value")
#overdose = overdose.drop(columns = ["Year", "Overdoses"]).groupby("County").sum().reset_index()

population = population.groupby("County").sum().reset_index()


#Convert to int
population['2018 Population'] = population['2018 Population'].astype(int)
population['2019 Population'] = population['2019 Population'].astype(int)
population['2020 Population'] = population['2020 Population'].astype(int)

population.head(100)

,County,2018 Population,2019 Population,2020 Population
0,Alamance,167407,169785,171980
1,Alexander,36664,36565,36444
2,Alleghany,10924,10919,10867
3,Anson,23075,21999,22030
4,Ashe,26591,26645,26577
...,...,...,...,...
95,Wayne,117851,117947,117333
96,Wilkes,66445,66189,65827
97,Wilson,78585,78991,78784
98,Yadkin,37251,37247,37187


In [26]:
#Combine pop with data
data["2018 Overdoses Per Capita"] = data["2018 Overdoses"] / pop["2018 Population"]
data["2019 Overdoses Per Capita"] = data["2019 Overdoses"] / pop["2019 Population"]
data["2020 Overdoses Per Capita"] = data["2020 Overdoses"] / pop["2020 Population"]
data = pd.merge(data, pop)
data = data.round(decimals = 3)


#Final Dataset
finalData = data
finalData.head(100)

,County,2018 Overdoses,2019 Overdoses,2020 Overdoses,2018 Per Capita Income,2019 Per Capita Income,2020 Per Capita Income,2018 Overdoses Per Capita,2019 Overdoses Per Capita,2020 Overdoses Per Capita,2018 Population,2019 Population,2020 Population
0,Alamance,2263,2231,2351,39742,41256,43973,0.014,0.013,0.014,167407,169785,171980
1,Alexander,642,432,459,36539,37758,40176,0.018,0.012,0.013,36664,36565,36444
2,Alleghany,137,120,165,36576,37987,40599,0.013,0.011,0.015,10924,10919,10867
3,Anson,459,303,444,35135,37741,38785,0.020,0.014,0.020,23075,21999,22030
4,Ashe,370,347,322,34791,36643,39612,0.014,0.013,0.012,26591,26645,26577
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Wayne,1613,2054,1658,38115,40116,42882,0.014,0.017,0.014,117851,117947,117333
96,Wilkes,1356,1158,1044,36445,37309,39518,0.020,0.017,0.016,66445,66189,65827
97,Wilson,817,988,1316,39158,40260,43111,0.010,0.013,0.017,78585,78991,78784
98,Yadkin,767,544,607,37328,39012,41250,0.021,0.015,0.016,37251,37247,37187
